# Phase II: Data Curation, Exploratory Analysis and Plotting (5\%)

### Team Members:
- Logan Lary
- Mark Tran
- Sabrina Valerjev

## Part 1: 
(1%) Expresses the central motivation of the project and explains the (at least) two key questions to be explored. Gives a summary of the data processing pipeline so a technical expert can easily follow along.

## Project Motivation 
The motivation for this project comes from our passion for movies and our curiosity about the factors that make them successful. As avid movie watchers, we are keen to explore the metadata behind films to uncover insights that intrigue us and hopefully resonate with others as well. This project aims to investigate the elements influencing both a movie's financial and critical success, such as cast, director, genre, release timing, studio, plot, and awards. By analyzing how these factors interact over time, we plan to create a predictive model that can estimate box office performance, forecast award nominations, and predict audience ratings. The key questions we seek to answer include: What factors have the most significant impact on a movie’s success? How do these factors evolve, and how can they be used to predict future outcomes? This research will uncover trends in popular genres and success patterns, offering valuable insights for investors and helping to guide movie recommendations based on predicted success.

## Summary of the Data Processing Pipeline
1. Web scrape from Box Office Mojo
2. Create list of movies per year
3. Use OMDb to access movie metadata each movie in the list
4. Merge all the data into a single dataframe
5. Clean the data

The first step in our data processing pipeline was to scrape all box office mojo data. For this step, we referrenced and modified exisiting code written and published by Justin Mitchel on GitHub. This code provided the base for understanding the intricate setup of Box Office Mojo. The code was modified to work on any valid inputted year. The data from Box Office Mojo provided us a list with all of the movies from each year. We isolated this list of movie titles and fed them into OMDb to collect the metadata on each film. The last step in data collection was to merge the data on the shared "Title" attribute. This ensures that the financial information and movie information both reference the correct film. One difficulty in this data collection process was the limited amount of calls we could make to OMDb per day. To get around this limitation, we only requested data for certain years each day, and merged the resulting dataframes to get the entire collection of data. The data cleaning of this dataframe was simple. We decided to simplify the financial data by removing the commas and dollars sign from the numbers and converting the sums from dollars to millions of dollars to make future math calculations more simple. We also dropped columns that we determined to be not needed.

## Part 2: 
(2\%) Obtains, cleans, and merges all data sources involved in the project.

In [3]:
# adding relevant imports
import requests
from requests_html import HTML
import json
import pathlib
import pandas as pd
import requests
from requests_html import HTML
from dataclasses import dataclass

In [4]:
# Source 1: Box Office Mojo
@dataclass
class ScrapeBoxOffice:
    base_endpoint:str = "https://www.boxofficemojo.com/year/world/"
    year:int = None
    save_raw:bool = False
    save:bool = False
    output_dir: str = "."
    table_selector: str = '.imdb-scroll-table'
    table_data = []
    table_header_names = []
    df = pd.DataFrame()
    
    @property
    def name(self):
        return self.year if isinstance(self.year, int) else 'world'
    
    def get_endpoint(self):
        endpoint = self.base_endpoint
        if isinstance(self.year, int):
            endpoint = f"{endpoint}{self.year}/"
        return endpoint
    
    def get_output_dir(self):
        return pathlib.Path(self.output_dir)
    
    def extract_html_str(self, endpoint=None):
        url = endpoint if endpoint is not None else self.get_endpoint()
        r = requests.get(url, stream=True)
        html_text = None
        status = r.status_code
        if r.status_code == 200:
            html_text = r.text
            if self.save_raw:
                output_fname = f"{self.name}.html"
                raw_output_dir = self.get_output_dir() / 'html'
                raw_output_dir.mkdir(exist_ok=True, parents=True)
                output_fname = raw_output_dir / output_fname
                with open(f"{output_fname}", 'w') as f:
                    f.write(html_text)
            return html_text, status
        return html_text, status
    
    def parse_html(self, html_str=''):
        r_html = HTML(html=html_str)
        r_table = r_html.find(self.table_selector)
        if len(r_table) == 0:
            return None
        table_data = []
        header_names = []
        parsed_table = r_table[0]
        rows = parsed_table.find("tr")
        header_row = rows[0]
        header_cols = header_row.find('th')
        header_names = [x.text for x in header_cols]
        for row in rows[1:]:
            cols = row.find("td")
            row_data = []
            row_dict_data = {}
            for i, col in enumerate(cols):
                header_name = header_names[i]
                row_data.append(col.text)
            table_data.append(row_data)
        self.table_data = table_data
        self.table_header_names = header_names
        return self.table_data, self.table_header_names
    
    def to_df(self, data=[], columns=[]):
        return pd.DataFrame(data, columns=columns)
    
    def run(self, save=False):
        save = self.save if save is False else save
        endpoint = self.get_endpoint()
        html_str, status = self.extract_html_str(endpoint=endpoint)
        if status not in range(200, 299):
            raise Exception(f"Extraction failed, endpoint status {status} at {endpoint}")
        data, headers = self.parse_html(html_str if html_str is not None else '')
        df = self.to_df(data=data, columns=headers)
        self.df = df
        if save:
            filepath = self.get_output_dir() / f'{self.name}.csv'
            df.to_csv(filepath, index=False)
        return self.df

In [ ]:
# Source 2: OMDb
API_KEY = "f3eb77a3"
URL = "http://www.omdbapi.com/?t="

def get_movie_data(url, movie):
    ''' Takes in the name of a movie and returns associated data on the movie.'''
    movie_link = process_movie_name(movie)
    complete_url = url + movie_link + "&apikey=" + API_KEY
    response = requests.get(complete_url) 
    return response.json()

def process_movie_name(movie):
    ''' Takes in the name of a movie and modifies it so that it can be used in API call.'''
    words = movie.split()
    return '+'.join(words)

# get the list of all movies in a year
# get data on all those movies
# save to a json
def get_year_movie_data(movie_titles, url, year):
    empty_data = {}
    data_list = []
    for movie in movie_titles:
        response = get_movie_data(url, movie)
        data_list.append(response)
    with open("MovieData" + year + ".json", 'w') as json_file:
        json.dump(data_list, json_file, indent=4) 

year = 2010
dataframe_1 = pd.DataFrame()
# we can only download ~ 3 years of data at one time
# only run each while loop once, to download all the data and prevent overloading the api
while year < 2013:
    scrapper = ScrapeBoxOffice(year=year, save=True, save_raw=True, output_dir='data')
    df_box = scrapper.run()
    movies_year = df_box["Release Group"].tolist()
    get_year_movie_data(movies_year, "http://www.omdbapi.com/?t=", str(year))
    file_path_movie = "MovieData" + str(year) + ".json"
    df_movie_data = pd.read_json(file_path_movie)
    box_df_bet = df_box.rename(columns={"Release Group": 'Title'})
    master_df = pd.merge(df_movie_data, box_df_bet, on = "Title", how = "inner")
    master_df["Year"] = year
    dataframe_1 = pd.concat([dataframe_1, master_df])
    year = year + 1

In [ ]:
# run the second day
year = 2013
dataframe_2 = pd.DataFrame()
while year < 2016:
    scrapper = ScrapeBoxOffice(year=year, save=True, save_raw=True, output_dir='data')
    df_box = scrapper.run()
    movies_year = df_box["Release Group"].tolist()
    get_year_movie_data(movies_year, "http://www.omdbapi.com/?t=", str(year))
    file_path_movie = "MovieData" + str(year) + ".json"
    df_movie_data = pd.read_json(file_path_movie)
    box_df_bet = df_box.rename(columns={"Release Group": 'Title'})
    master_df = pd.merge(df_movie_data, box_df_bet, on = "Title", how = "inner")
    master_df["Year"] = year
    dataframe_2 = pd.concat([dataframe_2, master_df])
    year = year + 1

In [ ]:
# run the third day
year = 2016
dataframe_3 = pd.DataFrame()
while year < 2020:
    scrapper = ScrapeBoxOffice(year=year, save=True, save_raw=True, output_dir='data')
    df_box = scrapper.run()
    movies_year = df_box["Release Group"].tolist()
    get_year_movie_data(movies_year, "http://www.omdbapi.com/?t=", str(year))
    file_path_movie = "MovieData" + str(year) + ".json"
    df_movie_data = pd.read_json(file_path_movie)
    box_df_bet = df_box.rename(columns={"Release Group": 'Title'})
    master_df = pd.merge(df_movie_data, box_df_bet, on = "Title", how = "inner")
    master_df["Year"] = year
    dataframe_3 = pd.concat([dataframe_3, master_df])
    year = year + 1

In [ ]:
# run the fourth day
year = 2020
dataframe_4 = pd.DataFrame()
while year < 2023:
    scrapper = ScrapeBoxOffice(year=year, save=True, save_raw=True, output_dir='data')
    df_box = scrapper.run()
    movies_year = df_box["Release Group"].tolist()
    get_year_movie_data(movies_year, "http://www.omdbapi.com/?t=", str(year))
    file_path_movie = "MovieData" + str(year) + ".json"
    df_movie_data = pd.read_json(file_path_movie)
    box_df_bet = df_box.rename(columns={"Release Group": 'Title'})
    master_df = pd.merge(df_movie_data, box_df_bet, on = "Title", how = "inner")
    master_df["Year"] = year
    dataframe_4 = pd.concat([dataframe_4, master_df])
    year = year + 1

In [ ]:
# merge all the dataframes
dataframe = pd.concat([dataframe_1, dataframe_2, dataframe_3, dataframe_4])

In [ ]:
# cleaning the data
def clean_box_office(df):
    '''Cleans box office sales by removing dollar signs and commas, and drops rows where Domestic value is "-".'''
    # Clean Worldwide column
    df = df[df["Domestic"] != "-"]
    df = df.dropna(subset = ["Worldwide", "Domestic", "Foreign"])
    df["Worldwide"] = (
        df["Worldwide"]
        .astype(str)  
        .str.replace("$", "", regex=False)  
        .str.replace(",", "", regex=False)  
        .astype(int)
    )
    # Clean Domestic column
    df["Domestic"] = (
        df["Domestic"]
        .astype(str)  
        .str.replace("$", "", regex=False) 
        .str.replace(",", "", regex=False)  
    
    )
    # Clean Foreign column
    df["Foreign"] = (
        df["Foreign"]
        .astype(str)  
        .str.replace("$", "", regex=False)  
        .str.replace(",", "", regex=False)  
    )
    # Creating new columns because the raw numbers are too large to process
    df["Worldwide_millions"] = pd.to_numeric(df["Worldwide"]) / 1000000
    df["Domestic_millions"] = pd.to_numeric(df["Domestic"]) / 1000000
    df["Foreign_millions"] = pd.to_numeric(df["Foreign"], errors="coerce") / 1000000
    return df

#dataframe = dataframe.drop(["Type", "Poster", "DVD", "totalSeasons", "Error", "Response", "Website", "Rank", "Production"], axis=1)
#cleaned_df = clean_box_office(dataframe)

## Part 3:
(2\%) Builds at least two visualizations (graphs/plots) from the data which help to understand or answer the questions of interest. These visualizations will be graded based on how much information they can effectively communicate to readers. Please make sure your visualization are sufficiently distinct from each other.

The visualizations that we made for this part are:
(1) Genre analysis histogram: for all movies in the time period, how many are there of each genre? Which genre is the most popular?
(2) Time series analysis on box office revenue: how has total box office revenue fluctuated over the time period?